In [2]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import datetime as dt
import pickle

In [3]:
# Create a df with holidays
from workalendar.europe import Belgium
cal = Belgium()
years = list(range(2014, 2021))
holidays = []
for year in years:
    holidays.extend(cal.holidays(year))
holidays

[(datetime.date(2014, 1, 1), 'New year'),
 (datetime.date(2014, 4, 21), 'Easter Monday'),
 (datetime.date(2014, 5, 1), 'Labour Day'),
 (datetime.date(2014, 5, 29), 'Ascension Thursday'),
 (datetime.date(2014, 6, 9), 'Whit Monday'),
 (datetime.date(2014, 7, 21), 'National Day'),
 (datetime.date(2014, 8, 15), 'Assumption of Mary to Heaven'),
 (datetime.date(2014, 11, 1), 'All Saints Day'),
 (datetime.date(2014, 11, 11), 'Armistice of 1918'),
 (datetime.date(2014, 12, 25), 'Christmas Day'),
 (datetime.date(2015, 1, 1), 'New year'),
 (datetime.date(2015, 4, 6), 'Easter Monday'),
 (datetime.date(2015, 5, 1), 'Labour Day'),
 (datetime.date(2015, 5, 14), 'Ascension Thursday'),
 (datetime.date(2015, 5, 25), 'Whit Monday'),
 (datetime.date(2015, 7, 21), 'National Day'),
 (datetime.date(2015, 8, 15), 'Assumption of Mary to Heaven'),
 (datetime.date(2015, 11, 1), 'All Saints Day'),
 (datetime.date(2015, 11, 11), 'Armistice of 1918'),
 (datetime.date(2015, 12, 25), 'Christmas Day'),
 (datetime.dat

In [4]:
df_holidays = pd.DataFrame(holidays, columns=['Datetime', 'holiday'])
df_holidays.head()
df_holidays.to_pickle('./holidays2014_2020.pkl')

In [5]:
df_holidays['Datetime'] = pd.to_datetime(df_holidays['Datetime'])
df_holidays['date'] = df_holidays['Datetime'].dt.date
df_holidays

,Datetime,holiday,date
0,2014-01-01,New year,2014-01-01
1,2014-04-21,Easter Monday,2014-04-21
2,2014-05-01,Labour Day,2014-05-01
3,2014-05-29,Ascension Thursday,2014-05-29
4,2014-06-09,Whit Monday,2014-06-09
...,...,...,...
65,2020-07-21,National Day,2020-07-21
66,2020-08-15,Assumption of Mary to Heaven,2020-08-15
67,2020-11-01,All Saints Day,2020-11-01
68,2020-11-11,Armistice of 1918,2020-11-11


In [6]:
data1 = pd.read_pickle('Building-1-Roulart Media Group/building_1_cnsumptions.pkl')
data1.index = pd.to_datetime(data1.index)
data1['Datetime'] = data1.index
data1

,Valeur,Datetime
Date,,
2014-01-06 02:00:00,6.51,2014-01-06 02:00:00
2014-01-06 02:15:00,6.51,2014-01-06 02:15:00
2014-01-06 02:30:00,6.72,2014-01-06 02:30:00
2014-01-06 02:45:00,6.51,2014-01-06 02:45:00
2014-01-06 03:00:00,6.93,2014-01-06 03:00:00
...,...,...
2020-06-30 23:00:00,5.04,2020-06-30 23:00:00
2020-06-30 23:15:00,3.36,2020-06-30 23:15:00
2020-06-30 23:30:00,3.36,2020-06-30 23:30:00


In [7]:
# Making sure there are no duplicated data
# If there are some duplicates we average the data during those duplicated days
data1 = data1.groupby('Datetime', as_index=False)['Valeur'].mean()

# Sorting the values
data1.sort_values('Datetime', inplace=True)
data1['Datetime'] = pd.to_datetime(data1['Datetime'])
data1['date'] = data1['Datetime'].dt.date
data1.head()

,Datetime,Valeur,date
0,2014-01-06 02:00:00,6.51,2014-01-06
1,2014-01-06 02:15:00,6.51,2014-01-06
2,2014-01-06 02:30:00,6.72,2014-01-06
3,2014-01-06 02:45:00,6.51,2014-01-06
4,2014-01-06 03:00:00,6.93,2014-01-06


In [8]:
dataplus = data1.merge(df_holidays, on='date', how='left')
dataplus['date'] = pd.to_datetime(dataplus['date'])
dataplus = dataplus.drop('Datetime_y', axis=1)
dataplus.columns = ['Datetime', 'Valeur', 'date', 'holiday']
dataplus.loc[dataplus['date'] >= '2014-06-09']

,Datetime,Valeur,date,holiday
3640,2014-06-09 00:00:00,6.93,2014-06-09,Whit Monday
3641,2014-06-09 00:15:00,6.72,2014-06-09,Whit Monday
3642,2014-06-09 00:30:00,6.93,2014-06-09,Whit Monday
3643,2014-06-09 00:45:00,7.77,2014-06-09,Whit Monday
3644,2014-06-09 01:00:00,7.35,2014-06-09,Whit Monday
...,...,...,...,...
213276,2020-12-06 22:45:00,5.88,2020-12-06,NaN
213277,2020-12-06 23:00:00,4.83,2020-12-06,NaN
213278,2020-12-06 23:15:00,3.36,2020-12-06,NaN
213279,2020-12-06 23:30:00,3.36,2020-12-06,NaN


In [9]:
dataplus['holiday'] = dataplus['holiday'].fillna(value=False)
dataplus.loc[dataplus['Datetime'] >= '2014-06-09']

,Datetime,Valeur,date,holiday
3640,2014-06-09 00:00:00,6.93,2014-06-09,Whit Monday
3641,2014-06-09 00:15:00,6.72,2014-06-09,Whit Monday
3642,2014-06-09 00:30:00,6.93,2014-06-09,Whit Monday
3643,2014-06-09 00:45:00,7.77,2014-06-09,Whit Monday
3644,2014-06-09 01:00:00,7.35,2014-06-09,Whit Monday
...,...,...,...,...
213276,2020-12-06 22:45:00,5.88,2020-12-06,False
213277,2020-12-06 23:00:00,4.83,2020-12-06,False
213278,2020-12-06 23:15:00,3.36,2020-12-06,False
213279,2020-12-06 23:30:00,3.36,2020-12-06,False


In [10]:
dataplus['working day'] =  dataplus['Datetime'].apply(cal.is_working_day)
dataplus.head()

,Datetime,Valeur,date,holiday,working day
0,2014-01-06 02:00:00,6.51,2014-01-06,False,True
1,2014-01-06 02:15:00,6.51,2014-01-06,False,True
2,2014-01-06 02:30:00,6.72,2014-01-06,False,True
3,2014-01-06 02:45:00,6.51,2014-01-06,False,True
4,2014-01-06 03:00:00,6.93,2014-01-06,False,True


In [11]:
dataplus = dataplus.drop('holiday', axis=1)
dataplus.head()

,Datetime,Valeur,date,working day
0,2014-01-06 02:00:00,6.51,2014-01-06,True
1,2014-01-06 02:15:00,6.51,2014-01-06,True
2,2014-01-06 02:30:00,6.72,2014-01-06,True
3,2014-01-06 02:45:00,6.51,2014-01-06,True
4,2014-01-06 03:00:00,6.93,2014-01-06,True


In [12]:
weather_rad = pd.read_csv('./weather_data/50.80_4.30_radiation.csv', parse_dates=[0])
weather_temp = pd.read_csv('./weather_data/50.80_4.30_temp.csv', parse_dates=[0])
weather_wind = pd.read_csv('./weather_data/50.80_4.30_wind_speed_10m.csv', parse_dates=[0])
weather_rad.head()

,time,Radiation (Wh/m²)
0,2014-01-01 00:00:00,0.000000
1,2014-01-01 01:00:00,0.000069
2,2014-01-01 02:00:00,0.000000
3,2014-01-01 03:00:00,0.000000
4,2014-01-01 04:00:00,0.000000


In [13]:
weather = weather_rad.merge(weather_temp, on='time', how='left')
weather = weather.merge(weather_wind, on='time', how='left')

In [14]:
weather.head()

,time,Radiation (Wh/m²),Temperatures (°C),Wind speed (m/s)
0,2014-01-01 00:00:00,0.000000,6.534021,1.775049
1,2014-01-01 01:00:00,0.000069,6.450159,1.750478
2,2014-01-01 02:00:00,0.000000,6.074030,1.791114
3,2014-01-01 03:00:00,0.000000,5.751886,1.805243
4,2014-01-01 04:00:00,0.000000,5.327020,1.875873


In [15]:
weather['time'] = pd.to_datetime(weather['time'])
weather = weather.set_index(['time'])
weather.head()

,Radiation (Wh/m²),Temperatures (°C),Wind speed (m/s)
time,,,
2014-01-01 00:00:00,0.000000,6.534021,1.775049
2014-01-01 01:00:00,0.000069,6.450159,1.750478
2014-01-01 02:00:00,0.000000,6.074030,1.791114
2014-01-01 03:00:00,0.000000,5.751886,1.805243
2014-01-01 04:00:00,0.000000,5.327020,1.875873


In [16]:
weather15 = weather.resample('15T').pad()

In [17]:
weather15['Datetime'] = weather15.index
weather15.head()

,Radiation (Wh/m²),Temperatures (°C),Wind speed (m/s),Datetime
time,,,,
2014-01-01 00:00:00,0.000000,6.534021,1.775049,2014-01-01 00:00:00
2014-01-01 00:15:00,0.000000,6.534021,1.775049,2014-01-01 00:15:00
2014-01-01 00:30:00,0.000000,6.534021,1.775049,2014-01-01 00:30:00
2014-01-01 00:45:00,0.000000,6.534021,1.775049,2014-01-01 00:45:00
2014-01-01 01:00:00,0.000069,6.450159,1.750478,2014-01-01 01:00:00


In [18]:
data_B1 = dataplus.merge(weather15, on='Datetime')
data_B1

,Datetime,Valeur,date,working day,Radiation (Wh/m²),Temperatures (°C),Wind speed (m/s)
0,2014-01-06 02:00:00,6.51,2014-01-06,True,0.000000,9.508508,3.071947
1,2014-01-06 02:15:00,6.51,2014-01-06,True,0.000000,9.508508,3.071947
2,2014-01-06 02:30:00,6.72,2014-01-06,True,0.000000,9.508508,3.071947
3,2014-01-06 02:45:00,6.51,2014-01-06,True,0.000000,9.508508,3.071947
4,2014-01-06 03:00:00,6.93,2014-01-06,True,0.000000,9.902185,2.997679
...,...,...,...,...,...,...,...
209821,2020-06-30 23:00:00,5.04,2020-06-30,True,0.000000,18.468164,1.596285
209822,2020-06-30 23:15:00,3.36,2020-06-30,True,0.000000,18.468164,1.596285
209823,2020-06-30 23:30:00,3.36,2020-06-30,True,0.000000,18.468164,1.596285
209824,2020-06-30 23:45:00,4.20,2020-06-30,True,0.000000,18.468164,1.596285


In [19]:
data_B1 = data_B1.drop('date', axis=1)

In [20]:
data_B1.to_csv('data_weather&holidays.csv')
data_B1.head()

,Datetime,Valeur,working day,Radiation (Wh/m²),Temperatures (°C),Wind speed (m/s)
0,2014-01-06 02:00:00,6.51,True,0.0,9.508508,3.071947
1,2014-01-06 02:15:00,6.51,True,0.0,9.508508,3.071947
2,2014-01-06 02:30:00,6.72,True,0.0,9.508508,3.071947
3,2014-01-06 02:45:00,6.51,True,0.0,9.508508,3.071947
4,2014-01-06 03:00:00,6.93,True,0.0,9.902185,2.997679
